# Team Ernst Engel Nutritional Adequacy Notebook
In this notebook, we examine the nutritional adequacy of the diets of the housholds in our analysis.

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets
import fooddatacentral as fdc

## Examing Nutritional Adequacy

In [103]:
country = 'uganda'
year_range = '2019-20'
diet_refs = pd.read_csv('./data/us_diet_minimums.csv')
dr_categories = diet_refs['Nutrition'].values
fct = pd.read_csv(f'./data/{country}_fct.csv')#.set_index('n')
food_prices = pd.read_csv(f'./data/{country}_food_prices.csv').fillna(0)
hh_chars = pd.read_csv(f'./data/{country}_hh_chars.csv')
### Deals With Housholds from an  Unknown Region
hh_chars['m'] = hh_chars['m'].fillna('Unknown')
expenditures = pd.read_csv(f'./data/{country}_expenditures_{year_range[-5:]}.csv').fillna(0)

In [88]:
hhc_sub = hh_chars[hh_chars['t'] == year_range].reset_index(drop = True)
fp_sub = food_prices[food_prices['t'] == year_range].set_index('m').drop(columns = ['t'])
food_cols = expenditures.iloc[0:2, 3:].columns
fp_sub_avgs = fp_sub.reset_index(drop = True)
fp_sub_avgs = pd.DataFrame(fp_sub_avgs.mean()).rename(columns = {0 : 'Mean_Price'})

In [90]:
def get_col_counts(expenditures_df, food_col, prices, price_avs):
    counts = []
    for idx in expenditures_df.index:
        region = expenditures_df.loc[idx, 'm']
        expenditure = expenditures_df.loc[idx, food_col]
        if region == 'Unknown':
            # Imputes price from unknown region with the mean accross all regions
            price = fp_sub_avgs.loc[food_col][0]
            if price == 0: # Don't want to divide by zero
                count = 0
                counts.append(count)
            else:
                count = expenditure / price
                counts.append(count)   
        else:
            price = prices.loc[region, food_col]
            if price == 0: # Don't want to divide by zero
                count = 0
                counts.append(count)
            else:
                count = expenditure / price
                counts.append(count)
    return counts

In [48]:
def get_counts(expenditures_df, food_cols, prices, price_avs):
    count_df = expenditures_df.drop(columns = food_cols)
    for food_col in food_cols:
        counts = get_col_counts(expenditures_df, food_col, prices, price_avs)
        count_df[food_col] = counts
    return count_df

In [50]:
exp_counts = get_counts(expenditures, food_cols, fp_sub, fp_sub_avgs)

In [54]:
household_master = hhc_sub.merge(exp_counts, left_on = ['i', 't', 'm'], right_on = ['i', 't', 'm'])

In [56]:
hhc_sub_num = hhc_sub.set_index('i').iloc[:, 2:-1]
hhc_sub_num

,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,M 00-03,M 04-08,M 09-13,M 14-18,M 19-30,M 31-50,M 51+
i,,,,,,,,,,,,,,
00c9353d8ebe42faabf5919b81d7fae7,1.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
062da72d5d3a457e9336b62c8bb9096d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0d0e29faff394154a69562b4527b48b8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
0e03e253c35d4333a1ffad2df9d38850,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1013000201,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c33f6cb57d9849949e08a7350dabb829,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d10a687889de469687377204195f3db0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
d24fa50d02c041969a42102d8ebdadc9,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [74]:
# If expenditures weekly, extrapolation_constant = 7
# If expenditures daily, extrapolation_constant = 1
extrapolation_constant = 7
diet_refs_rev = diet_refs.set_index('Nutrition').drop(columns = ['Source']).rename(columns = {'C 1-3' : 'F 00-03', 
                                                                                              'F 4-8' : 'F 04-08',
                                                                                              'M 4-8' : 'M 04-08',
                                                                                              'F 9-13' : 'F 09-13',
                                                                                              'M 9-13' : 'M 09-13'})
diet_refs_rev['M 00-03'] = diet_refs_rev['F 00-03']
diet_refs_rev = diet_refs_rev[hhc_sub_num.columns]
diet_refs_rev = extrapolation_constant * diet_refs_rev
diet_refs_rev

,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,M 00-03,M 04-08,M 09-13,M 14-18,M 19-30,M 31-50,M 51+
Nutrition,,,,,,,,,,,,,,
Energy,7000.0,8400.0,11200.0,12600.0,14000.0,12600.0,11200.0,7000.0,9800.0,12600.0,15400.0,16800.0,15400.0,14000.0
Protein,91.0,133.0,238.0,322.0,322.0,322.0,322.0,91.0,133.0,238.0,364.0,392.0,392.0,392.0
"Fiber, total dietary",98.0,117.6,156.8,176.4,196.0,176.4,156.8,98.0,137.2,176.4,215.6,235.2,215.6,196.0
"Folate, DFE",1050.0,1400.0,2100.0,2800.0,2800.0,2800.0,2800.0,1050.0,1400.0,2100.0,2800.0,2800.0,2800.0,2800.0
"Calcium, Ca",4900.0,7000.0,9100.0,9100.0,7000.0,7000.0,8400.0,4900.0,7000.0,9100.0,9100.0,7000.0,7000.0,7000.0
"Carbohydrate, by difference",910.0,910.0,910.0,910.0,910.0,910.0,910.0,910.0,910.0,910.0,910.0,910.0,910.0,910.0
"Iron, Fe",49.0,70.0,56.0,105.0,126.0,126.0,56.0,49.0,70.0,56.0,77.0,56.0,56.0,56.0
"Magnesium, Mg",560.0,910.0,1680.0,2520.0,2170.0,2240.0,2240.0,560.0,910.0,1680.0,2870.0,2800.0,2940.0,2940.0
Niacin,42.0,56.0,84.0,98.0,98.0,98.0,98.0,42.0,56.0,84.0,112.0,112.0,112.0,112.0


In [84]:
hh_diet_reqs = hhc_sub_num @ diet_refs_rev.T
hh_diet_reqs

Nutrition,Energy,Protein,"Fiber, total dietary","Folate, DFE","Calcium, Ca","Carbohydrate, by difference","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P","Potassium, K",Riboflavin,Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn"
i,,,,,,,,,,,,,,,,,,,,
00c9353d8ebe42faabf5919b81d7fae7,82600.0,1939.0,1156.4,17150.0,56700.0,6370.0,602.0,14210.0,616.0,48020.0,217000.0,46.2,46.2,30800.0,102.9,55.3,2835.0,644.0,3465.0,378.0
062da72d5d3a457e9336b62c8bb9096d,15400.0,392.0,215.6,2800.0,7000.0,910.0,56.0,2940.0,112.0,4900.0,32900.0,9.1,8.4,6300.0,16.8,9.1,630.0,105.0,840.0,77.0
0d0e29faff394154a69562b4527b48b8,43400.0,896.0,607.6,7700.0,23800.0,3640.0,280.0,6230.0,294.0,16240.0,107800.0,23.8,23.1,15400.0,46.2,25.2,1365.0,294.0,1890.0,175.0
0e03e253c35d4333a1ffad2df9d38850,56000.0,1176.0,784.0,10150.0,35000.0,4550.0,357.0,8260.0,392.0,25270.0,144900.0,30.8,30.1,20300.0,60.9,32.9,1750.0,378.0,2485.0,245.0
1013000201,11200.0,322.0,156.8,2800.0,8400.0,910.0,56.0,2240.0,98.0,4900.0,32900.0,7.7,7.7,4900.0,16.8,10.5,525.0,105.0,630.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c33f6cb57d9849949e08a7350dabb829,11200.0,322.0,156.8,2800.0,8400.0,910.0,56.0,2240.0,98.0,4900.0,32900.0,7.7,7.7,4900.0,16.8,10.5,525.0,105.0,630.0,56.0
d10a687889de469687377204195f3db0,42000.0,1078.0,588.0,8400.0,24500.0,2730.0,189.0,8050.0,322.0,18550.0,98700.0,25.9,24.5,17500.0,50.4,28.7,1680.0,315.0,1995.0,210.0
d24fa50d02c041969a42102d8ebdadc9,95200.0,2128.0,1332.8,17850.0,60200.0,7280.0,574.0,15330.0,686.0,47670.0,242200.0,53.2,51.8,35700.0,107.1,60.2,3150.0,665.0,4270.0,441.0


In [63]:
hh_consumption_cols = list(food_cols.to_numpy())
hh_consumption_cols.append('i')

hh_consumption = household_master[hh_consumption_cols].set_index('i')
hh_consumption

,Beans,Beef,Beer,Biscuits,Bongo,Bread,"Butter, etc.",Cabbages,Cake,Cassava,...,Sugarcane,Sweet Bananas,Sweet Potatoes,Tea,Tomatoes,Waragi,Water,Wheat (flour),Yam,Yogurt
i,,,,,,,,,,,,,,,,,,,,,
00c9353d8ebe42faabf5919b81d7fae7,1.582418,0.00,0.000000,0.0,0.0,0.000000,0.0,0.833636,0.0,4.857143,...,0.0,0.000000,5.034965,0.020000,1.600000,0.0,0.0,0.0,0.865775,0.0
062da72d5d3a457e9336b62c8bb9096d,0.000000,0.00,0.000000,0.0,0.0,0.111111,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
0d0e29faff394154a69562b4527b48b8,0.439560,0.45,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,3.238095,...,0.0,0.000000,0.000000,0.000000,1.280000,0.0,0.0,0.0,0.000000,0.0
0e03e253c35d4333a1ffad2df9d38850,1.230769,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,2.590476,...,0.0,0.000000,6.293706,0.020000,0.800000,0.0,0.0,0.0,0.000000,0.0
1013000201,0.000000,0.00,2.916667,0.0,0.0,0.000000,0.0,0.000000,0.0,2.857143,...,0.0,1.709975,0.000000,0.030769,3.126136,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bfdf0d66403440ceab439b1e1c47cdea,0.527473,1.00,0.000000,0.0,2.8,0.000000,0.0,0.297727,0.0,2.590476,...,0.0,0.000000,7.552448,0.010000,1.600000,0.0,0.0,0.0,0.288592,0.0
c33f6cb57d9849949e08a7350dabb829,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,1.785714,...,0.0,0.000000,1.290323,0.015385,1.488636,0.0,0.0,0.0,0.000000,0.0
d10a687889de469687377204195f3db0,0.909091,0.00,0.000000,0.0,0.0,0.000000,0.0,0.425325,0.0,0.000000,...,0.0,0.000000,2.720000,0.000000,1.333333,0.0,0.0,0.0,0.000000,0.0


In [113]:
len(fct[fct['j'].isin(hh_consumption.columns)])

33